In [7]:
!brew install lightgbm

Updating Homebrew...
==> Downloading https://homebrew.bintray.com/bottles-portable-ruby/portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
######################################################################### 100.0%
==> Pouring portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
acl2                       htmltest                   oras
act                        httpx                      ormolu
aerc                       hy                         ory-hydra
aida-header                i686-elf-binutils          osi
aliddns                    i686-elf-gcc               osm
alsa-lib                   iconsur                    osmcoastline
amp                        idris2                     overdrive
ansible@2.9                indicators                 ox
apidoc                     infracost                  packetbeat
arb                        inframap                   pandoc-include-code
argo                       


==> Downloading https://homebrew.bintray.com/bottles/libomp-11.1.0.catalina.bott
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/45a5aa653bd45bd5ff585
######################################################################## 100.0%
==> Downloading https://homebrew.bintray.com/bottles/lightgbm-3.1.1.catalina.bot
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/ff1a50ff028ff22997b0a
######################################################################## 100.0%
==> Installing dependencies for lightgbm: libomp
==> Installing lightgbm dependency: libomp
==> Pouring libomp-11.1.0.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/libomp/11.1.0: 9 files, 1.4MB
==> Installing lightgbm
==> Pouring lightgbm-3.1.1.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/lightgbm/3.1.1: 89 files, 17.6MB
==> `brew cleanup` has not been run in 30 days, running now...
Removing: /Users/daksh_mac/Library/Caches/Homebrew/aom--2.0.0.catalina.bottle.tar.gz... (5.5MB)
Removing: /Users/daksh_mac/Library/C

Removing: /Users/daksh_mac/Library/Logs/Homebrew/gnuplot... (64B)
Removing: /Users/daksh_mac/Library/Logs/Homebrew/pango... (64B)
Removing: /Users/daksh_mac/Library/Logs/Homebrew/libcerf... (64B)
Removing: /Users/daksh_mac/Library/Logs/Homebrew/qt... (64B)
Removing: /Users/daksh_mac/Library/Logs/Homebrew/lua... (64B)
Removing: /Users/daksh_mac/Library/Logs/Homebrew/gd... (64B)
Pruned 2 symbolic links from /usr/local


In [98]:
import sys
import os
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

import lightgbm as lgb
import random

In [99]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

In [100]:
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.utils import shuffle

In [101]:
%ls data

180_days_maxT_24hrPrecip.csv
180_days_mean1hrT_1hrPrecip.csv
180_days_meanT_24hrPrecip.csv
2019-09-27-basel-collections.csv
2019-09-27-basel-image-metadata.csv
2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv
2019-09-27-basel-measures-FEAT-TempPrecip.csv
2019-09-27-basel-measures-FEAT.csv
2019-09-27-basel-measures-cleaned.csv
2019-09-27-basel-measures-prediction-cleaned-FEAT.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip-Twitter.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv
2019-09-27-basel-measures-prediction-cleaned.csv
2019-09-27-basel-measures-prediction.csv
2019-09-27-basel-measures.csv
all_tweets.csv
distributions/
event_cal.csv
twitter_day_features-TEST.csv
twitter_day_features.csv


## load training data

In [102]:
in_df_train = pd.read_csv("./data/2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv")

in_df_train = in_df_train.drop([c for c in in_df_train.columns if c.startswith('Unnamed')], axis=1)

in_df_train['comb_id'] = in_df_train[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

In [103]:
print(in_df_train.shape)
in_df_train.head()

(58242, 18)


,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_english,weekday_english,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,comb_id
0,1175332462,287,2019-04-01 8:40:49,Leimgrubenweg,bus_stop,3.20000,3.20000,8,0,4,April,Montag,morning,11.4,0.0,0,2,1175332462-287-2019-04-01 8:40:49
1,25149740,86_9668,2019-04-01 8:40:53,Leimgrubenweg,secondary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,11,2,25149740-86_9668-2019-04-01 8:40:53
2,117485263,86_11641,2019-04-01 8:40:53,Reinacherstrasse,primary,3.35764,3.35764,8,0,4,April,Montag,morning,11.4,0.0,8,2,117485263-86_11641-2019-04-01 8:40:53
3,2621172927,287,2019-04-01 8:40:57,Leimgrubenweg,bus_stop,3.19757,3.19757,8,0,4,April,Montag,morning,11.4,0.0,0,2,2621172927-287-2019-04-01 8:40:57
4,148833576,86_12436,2019-04-01 8:42:11,Dornacherstrasse,secondary,5.00000,5.00000,8,0,4,April,Montag,morning,11.4,0.0,11,2,148833576-86_12436-2019-04-01 8:42:11


## Converting the data to Indian context

In [104]:
temp_table=in_df_train
place_name_list_foreign=list(temp_table['place_name'])


In [105]:
replacing_place_names={}

unique_places=list(set(place_name_list_foreign))
num_unique_places=len(unique_places)

'''reading the delhi cities from a json file which is scraped from internet using Scrapy'''
    
places_dataframe=pd.read_json(r'/Users/daksh_mac/Desktop/SmartCleanCityHackZurich19-master/cityScraping/cityScraping/spiders/delhiPlaces.json')

places=list(places_dataframe['place'])

random.shuffle(places)

'''Now I have to replace names of the foreign cities with the Indian ones'''

for i in range(num_unique_places):
    replacing_place_names[unique_places[i]]=places[i]

for i in range(len(place_name_list_foreign)):
    
    place_name_list_foreign[i]=replacing_place_names[place_name_list_foreign[i]]

In [106]:
temp_table['place_name']=place_name_list_foreign
in_df_train['place_name']=place_name_list_foreign

## Fine Tuning The Day and The Month

In [107]:
english_months = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September',
                10:'October', 11:'November', 12:'December'}

english_weekdays = {0:'Monday', 1:'Tuesday', 2:'Wednesday',3:'Thursday',4:'Friday', 5:'Saturday', 6:'Sunday'}


In [108]:
month_num=list(in_df_train['month'])
weekday_num=list(in_df_train['weekday'])

orig_months=list(in_df_train['month_english'])
orig_weekday=list(in_df_train['weekday_english'])

for i in range(len(month_num)):
    
    orig_months[i]=english_months[month_num[i]]
    orig_weekday[i]=english_weekdays[weekday_num[i]]

in_df_train['month_english']=orig_months
in_df_train['weekday_english']=orig_weekday
    

## Adding Certain Parameters

In [109]:
import datetime
def get_daydate(date_str):
    '''Returns shortened date string in the format %Y-%m-%d.'''
    
    d = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
       
    return str(datetime.datetime.strftime(d, '%Y-%m-%d'))

in_df_train['day_date'] = in_df_train.date.apply(lambda x: get_daydate(x))

in_df_train.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_english,weekday_english,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,comb_id,day_date
0,1175332462,287,2019-04-01 8:40:49,Chhattarpur,bus_stop,3.20000,3.20000,8,0,4,April,Monday,morning,11.4,0.0,0,2,1175332462-287-2019-04-01 8:40:49,2019-04-01
1,25149740,86_9668,2019-04-01 8:40:53,Chhattarpur,secondary,3.35764,3.35764,8,0,4,April,Monday,morning,11.4,0.0,11,2,25149740-86_9668-2019-04-01 8:40:53,2019-04-01
2,117485263,86_11641,2019-04-01 8:40:53,Jānti Khurd,primary,3.35764,3.35764,8,0,4,April,Monday,morning,11.4,0.0,8,2,117485263-86_11641-2019-04-01 8:40:53,2019-04-01
3,2621172927,287,2019-04-01 8:40:57,Chhattarpur,bus_stop,3.19757,3.19757,8,0,4,April,Monday,morning,11.4,0.0,0,2,2621172927-287-2019-04-01 8:40:57,2019-04-01
4,148833576,86_12436,2019-04-01 8:42:11,Sadr Bāzār,secondary,5.00000,5.00000,8,0,4,April,Monday,morning,11.4,0.0,11,2,148833576-86_12436-2019-04-01 8:42:11,2019-04-01


In [110]:
in_df_all_train = in_df_train
in_df_all_train.head()

,osm_id,cci_id,date,place_name,place_type,cci,cci_p,hour,weekday,month,month_english,weekday_english,daytime,t_mean_2m_24h:C,precip_24h:mm,place_type_enc,daytime_enc,comb_id,day_date
0,1175332462,287,2019-04-01 8:40:49,Chhattarpur,bus_stop,3.20000,3.20000,8,0,4,April,Monday,morning,11.4,0.0,0,2,1175332462-287-2019-04-01 8:40:49,2019-04-01
1,25149740,86_9668,2019-04-01 8:40:53,Chhattarpur,secondary,3.35764,3.35764,8,0,4,April,Monday,morning,11.4,0.0,11,2,25149740-86_9668-2019-04-01 8:40:53,2019-04-01
2,117485263,86_11641,2019-04-01 8:40:53,Jānti Khurd,primary,3.35764,3.35764,8,0,4,April,Monday,morning,11.4,0.0,8,2,117485263-86_11641-2019-04-01 8:40:53,2019-04-01
3,2621172927,287,2019-04-01 8:40:57,Chhattarpur,bus_stop,3.19757,3.19757,8,0,4,April,Monday,morning,11.4,0.0,0,2,2621172927-287-2019-04-01 8:40:57,2019-04-01
4,148833576,86_12436,2019-04-01 8:42:11,Sadr Bāzār,secondary,5.00000,5.00000,8,0,4,April,Monday,morning,11.4,0.0,11,2,148833576-86_12436-2019-04-01 8:42:11,2019-04-01


### load test data

In [111]:
%ls ./data/

180_days_maxT_24hrPrecip.csv
180_days_mean1hrT_1hrPrecip.csv
180_days_meanT_24hrPrecip.csv
2019-09-27-basel-collections.csv
2019-09-27-basel-image-metadata.csv
2019-09-27-basel-measures-FEAT-TempPrecip-IntLabels.csv
2019-09-27-basel-measures-FEAT-TempPrecip.csv
2019-09-27-basel-measures-FEAT.csv
2019-09-27-basel-measures-cleaned.csv
2019-09-27-basel-measures-prediction-cleaned-FEAT.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip-Twitter.csv
2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv
2019-09-27-basel-measures-prediction-cleaned.csv
2019-09-27-basel-measures-prediction.csv
2019-09-27-basel-measures.csv
all_tweets.csv
distributions/
event_cal.csv
twitter_day_features-TEST.csv
twitter_day_features.csv


In [112]:
in_df_test2 = pd.read_csv("./data/2019-09-27-basel-measures-prediction-cleaned-FEAT.csv")

in_df_test2['comb_id'] = in_df_test2[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

in_df_test2 = in_df_test2.drop([c for c in in_df_test2.columns if c.startswith('Unnamed')], axis=1)

in_df_test2 = in_df_test2[['weekday', 'month', 'daytime', 'comb_id']]

print(in_df_test2.shape)

in_df_test2.head()

(7292, 4)


,weekday,month,daytime,comb_id
0,1,6,morning,172611027-86_12812-2019-06-25 6:25:49
1,1,6,morning,3828924874-260-2019-06-25 6:26:22
2,1,6,morning,23913163-86_9255-2019-06-25 6:26:26
3,1,6,morning,23913163-86_17476-2019-06-25 6:26:26
4,1,6,morning,144481239-86_12223-2019-06-25 6:27:21


In [113]:
in_df_test1 = pd.read_csv("./data/2019-09-27-basel-measures-prediction-cleaned-TempPrecip.csv")

in_df_test1['comb_id'] = in_df_test1[['osm_id', 'cci_id', 'date']].apply(lambda x: '-'.join([str(i) for i in x]), axis=1)

in_df_test1 = in_df_test1.drop([c for c in in_df_test1.columns if c.startswith('Unnamed')], axis=1)

in_df_test1['day_date'] = in_df_test1.date.apply(lambda x: get_daydate(x))

print(in_df_test1.shape)

in_df_test1.head()

(7292, 11)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date
0,172611027,86_12812,2019-06-25 6:25:49,Klingelbergstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25
1,3828924874,260,2019-06-25 6:26:22,Metzerstrasse,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25
2,23913163,86_9255,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25
3,23913163,86_17476,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25
4,144481239,86_12223,2019-06-25 6:27:21,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25


In [114]:
in_df_test1.comb_id.unique().shape

(7292,)

In [115]:
in_df_test = pd.merge(in_df_test1, in_df_test2, how='inner', on='comb_id')

print(in_df_test.shape)

in_df_test.head()

(7292, 14)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date,weekday,month,daytime
0,172611027,86_12812,2019-06-25 6:25:49,Klingelbergstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25,1,6,morning
1,3828924874,260,2019-06-25 6:26:22,Metzerstrasse,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25,1,6,morning
2,23913163,86_9255,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25,1,6,morning
3,23913163,86_17476,2019-06-25 6:26:26,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25,1,6,morning
4,144481239,86_12223,2019-06-25 6:27:21,Metzerstrasse,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25,1,6,morning


## Correcting The Places Name

In [116]:
place_name_list_foreign=list(in_df_test['place_name'])

replacing_place_names={}

unique_places=list(set(place_name_list_foreign))
num_unique_places=len(unique_places)

'''Now I have to replace names of the foreign cities with the Indian ones'''

for i in range(num_unique_places):
    replacing_place_names[unique_places[i]]=places[i+324]

for i in range(len(place_name_list_foreign)):
    
    place_name_list_foreign[i]=replacing_place_names[place_name_list_foreign[i]]


in_df_test['place_name']=place_name_list_foreign

In [117]:
in_df_all_test = in_df_test
print(in_df_all_test.shape)
in_df_all_test.head()

(7292, 14)


,osm_id,cci_id,date,place_name,place_type,date_short,validdate,t_mean_2m_24h:C,precip_24h:mm,comb_id,day_date,weekday,month,daytime
0,172611027,86_12812,2019-06-25 6:25:49,Nasīrpur,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,172611027-86_12812-2019-06-25 6:25:49,2019-06-25,1,6,morning
1,3828924874,260,2019-06-25 6:26:22,Raghubir Enclave,bus_stop,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,3828924874-260-2019-06-25 6:26:22,2019-06-25,1,6,morning
2,23913163,86_9255,2019-06-25 6:26:26,Raghubir Enclave,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_9255-2019-06-25 6:26:26,2019-06-25,1,6,morning
3,23913163,86_17476,2019-06-25 6:26:26,Raghubir Enclave,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,23913163-86_17476-2019-06-25 6:26:26,2019-06-25,1,6,morning
4,144481239,86_12223,2019-06-25 6:27:21,Raghubir Enclave,residential,2019-06-25,2019-06-25T07:00:00Z,24.7,0.0,144481239-86_12223-2019-06-25 6:27:21,2019-06-25,1,6,morning


In [118]:
def row_feat_dict(row):
    '''
    extract features for one row as a dictionary

    '''
    
    cols = ['place_type', 'weekday', 'month', 'daytime']

    feat_dict = {c:row[c] for c in cols}  # dictionary for one training instance
    
    if str(row['place_name']) == 'nan':
        feat_dict['place_name'] = 'undefined'
    else:
        feat_dict['place_name'] = row.place_name.lower()
    
    
    feat_dict['t_mean_2m_24h:C'] = row['t_mean_2m_24h:C']
    
    feat_dict['precip_24h:mm'] = row['precip_24h:mm']
    
        
    return feat_dict

In [119]:
feats_dict_list_train = in_df_all_train.apply(lambda x: row_feat_dict(x), axis=1)
feats_dict_list_test = in_df_all_test.apply(lambda x: row_feat_dict(x), axis=1)

In [120]:
dict_vec = DictVectorizer()

# fit vectorizer on training data
dict_vec.fit(feats_dict_list_train)

DictVectorizer()

In [121]:
list(dict_vec.get_feature_names())
feature_names = dict_vec.get_feature_names()
feature_names

['daytime=afternoon',
 'daytime=evening',
 'daytime=morning',
 'month',
 'place_name=adarshnagar',
 'place_name=alipur',
 'place_name=ambar hai',
 'place_name=ansarinagar',
 'place_name=arjunnugar',
 'place_name=ashoknagar',
 'place_name=asmara lines',
 'place_name=auchandi',
 'place_name=auhammadpur majri',
 'place_name=aulia masjid',
 'place_name=baba haridas enclave',
 'place_name=baba haridasnagar',
 'place_name=badarpur',
 'place_name=badosra',
 'place_name=bagong delhi',
 'place_name=bamnoli village',
 'place_name=baprola village',
 'place_name=basāl dārāpur',
 'place_name=bawana village',
 'place_name=bawāna',
 'place_name=begampur',
 'place_name=behlolpur bāngar',
 'place_name=betaista enclave',
 'place_name=bharatnagar',
 'place_name=bharthal village',
 'place_name=bhili bhulaiyan',
 'place_name=bholanathnagar',
 'place_name=bhūpendrāsinghnagar',
 'place_name=bijwasan village',
 'place_name=bijwāsan',
 'place_name=bindāpur',
 'place_name=brahmapuri',
 'place_name=burari',
 'pl

In [122]:
# transform training features to (dense) feature matrix

X_train = dict_vec.transform(feats_dict_list_train)
X_train = X_train.todense()
print(X_train.shape)


(58242, 344)


In [123]:
# transform test features to (dense) feature matrix

X_test = dict_vec.transform(feats_dict_list_test)
X_test = X_test.todense()
print(X_test.shape)

(7292, 344)


In [124]:
y_train = in_df_train.cci.values
y_train

array([3.2       , 3.35764015, 3.35764015, ..., 3.86769316, 4.39927712,
       4.39927712])

In [125]:
random_state = 98
#random_state = 72


pipelines = {
    
    'LinearRegression': LinearRegression(),
    #'DecisionTree': DecisionTreeClassifier(),
    #'GBC': GradientBoostingClassifier(random_state=random_state),
    'LGBMRegressor': lgb.LGBMRegressor(
                                    random_state=random_state,
                                    boosting_type='gbdt', 
                                    learning_rate=0.01,
                                    num_leaves=31,
                                    max_depth=-1,
                                    reg_lambda=0)  # with tuned hyperparams
    
}

In [126]:
#print(sorted(metrics.SCORERS.keys()))

def run_pipelines(X, y, pipelines, num_folds=10, random_state=7):
    '''Runs cross validation (k=num_folds) for all given pipelines'''
    
    regs = dict()
    
    X, y = shuffle(X, y, random_state=random_state)
    
    folds = num_folds

    for reg_name in pipelines:
        print(reg_name)
        reg = pipelines[reg_name]
        
        result = cross_validate(reg, X, y, scoring=['neg_mean_absolute_error'], cv=folds)
        
        print(result)

        reg = reg.fit(X,y)
        
        regs[reg_name] = reg
        
    return regs
        
               

In [127]:
regs = run_pipelines(X_train, y_train, pipelines)

LinearRegression
{'fit_time': array([1.04895997, 1.00579715, 1.04163289, 0.98905706, 1.01406002,
       0.94954491, 0.96157289, 0.96619272, 0.91369915, 0.91456699]), 'score_time': array([0.00452614, 0.00252581, 0.0020299 , 0.00205374, 0.00204015,
       0.00204206, 0.00200796, 0.002069  , 0.00212193, 0.00212979]), 'test_neg_mean_absolute_error': array([-0.52375039, -0.5321963 , -0.53657978, -0.52971111, -0.53152459,
       -0.53311631, -0.52847948, -0.52443607, -0.52718736, -0.52305106])}
LGBMRegressor
{'fit_time': array([0.38017201, 0.31926012, 0.32241607, 0.31846976, 0.31709123,
       0.36453295, 0.3208468 , 0.31894112, 0.33858275, 0.34248495]), 'score_time': array([0.00900197, 0.00826097, 0.00844884, 0.00835991, 0.0083189 ,
       0.00817895, 0.00833011, 0.00857496, 0.00891519, 0.009197  ]), 'test_neg_mean_absolute_error': array([-0.52307177, -0.53335073, -0.53766329, -0.53398119, -0.53407552,
       -0.5328943 , -0.52908465, -0.52734622, -0.52033064, -0.5255365 ])}


In [128]:
test_preds_lgbm = regs['LGBMRegressor'].predict(X_test)

test_out_lgbm = in_df_all_test[['osm_id','cci_id','date']].copy()
test_out_lgbm['cci'] = test_preds_lgbm

test_out_lgbm.to_csv('test-PREDS-lgbm.csv', index=None, header=True)

test_out_lgbm.cci.mean()

3.911610039485843

In [129]:
test_preds_linreg = regs['LinearRegression'].predict(X_test)

test_out_linreg = in_df_all_test[['osm_id','cci_id','date']].copy()
test_out_linreg['cci'] = test_preds_linreg

test_out_linreg.to_csv('test-PREDS-linreg.csv', index=None, header=True)

test_out_linreg.cci.mean()

3.833013077950757